In [1]:
import random
import copy
import multiprocessing
from joblib import Parallel, delayed  # joblib is an external package, must run 'pip install joblib' in console
from timeit import default_timer as timer  # for timing the loops
import shutil  # for combining files at the end
import collections # for easy summarizing of all player totals, see https://www.geeksforgeeks.org/python-sum-list-of-dictionaries-with-same-key/

In [2]:
# global_debug=True
hand_list = []

### set up the constants we'll need

In [3]:
card_values = { '2':2, '3':3, '4':4, '5':5, '6':6, '7':7, '8':8, '9':9, 'T':10, 'J':10, 'Q':10, 'K':10, 'A':11 }

suits = ['♠️', '♣️', '♥️', '♦️']  # ['s', 'c', 'h', 'd'] # { 's':'♠️', 'c':'♣️', 'h':'♥️', 'd':'♦️' }

paytable = {
    'player_blackjack' : 2.50,   # 3:2 = 2.50, 6:5 = 2.20, even = 2
    'player_win' : 2,
    'push' : 1,
    'player_loss' : 0
}

### define the doubling/splitting strategies players will follow

In [4]:
basic_split_strategy = {  2 : [2, 3, 4, 5, 6, 7] ,
                          3 : [2, 3, 4, 5, 6, 7] ,
                          4 : [5, 6] ,
                          5 : [ ] ,
                          6 : [2, 3, 4, 5, 6] , 
                          7 : [2, 3, 4, 5, 6, 7] ,
                          8 : [2, 3, 4, 5, 6, 7, 8, 9, 10, 11] ,
                          9 : [2, 3, 4, 5, 6, 8, 9] ,
                         10 : [ ] ,
                         11 : [2, 3, 4, 5, 6, 7, 8, 9, 10, 11] }

basic_double_strategy = {"hard": {  9 : [3, 4, 5, 6] ,
                                   10 : [2, 3, 4, 5, 6, 7, 8, 9] ,
                                   11 : [2, 3, 4, 5, 6, 7, 8, 9, 10, 11] }, 
                         "soft": { 13 : [5, 6] ,
                                   14 : [5, 6] ,
                                   15 : [4, 5, 6] ,
                                   16 : [4, 5, 6] ,
                                   17 : [3, 4, 5, 6] ,
                                   18 : [3, 4, 5, 6] } }

# "optimal" split strategy has numerous differences from basic
df_split_strategy = {     2 : [5, 6] ,
                          3 : [6] ,
                          4 : [3, 4, 5, 6] ,
                          5 : [ ] ,
                          6 : [2, 3, 4, 5, 6, 7, 8] , 
                          7 : [3, 4, 5, 6, 7] ,
                          8 : [2, 3, 4, 5, 6, 7, 8] ,
                          9 : [2, 3, 4, 5, 6, 7, 8] ,
                         10 : [ ] ,
                         11 : [2, 3, 4, 5, 6, 7, 8, 9, 10, 11] }

# same as basic except addition of 10 vs 10, and no soft doubling
df_double_strategy = {   "hard": {  9 : [3, 4, 5, 6] ,
                                   10 : [2, 3, 4, 5, 6, 7, 8, 9, 10] ,
                                   11 : [2, 3, 4, 5, 6, 7, 8, 9, 10, 11] }, 
                         "soft": { 13 : [] ,
                                   14 : [] ,
                                   15 : [] ,
                                   16 : [] ,
                                   17 : [] ,
                                   18 : [] } }

late_surrender_strategy = {}

In [5]:
class Card:
    def __init__(self, face, suit):
        self.exposed = False
        self.face = face
        self.suit = suit
        
    @property
    def value(self):
        return card_values[self.face] if self.exposed else 0
        
    def __repr__(self):
        return f"{self.face}{self.suit}" if self.exposed else f"▮"
    
    def expose(self):
        self.exposed = True

In [6]:
class Shoe: 
    
    def __init__(self, deck_count=6, minimum_cards=94):  # 94 cards = 70% penetration
        self.minimum_cards = minimum_cards
        self.full_deck = []
        for i in range(deck_count):
            # must recreate the deck every time otherwise Python will see all same cards as same object!
            for card in [ Card(face, suit) for suit in suits for face in list(card_values.keys()) ]:
                card.exposed = False
                self.full_deck.append(card)
        self.cards = []                
        self.shuffle()
        
    def __repr__(self):
        return str(self.cards)
    
    def shuffle(self):
        self.cards.clear()
        for c in (self.full_deck):
            self.cards.append(copy.copy(c))
        random.shuffle(self.cards)

    
    def shuffle_check(self):
        if len(self.cards) < self.minimum_cards:
            self.shuffle()
    
    def pitch(self, hand, expose=True, printme=False):
        new_card = self.cards.pop(0)  # pop(0) is first card in list, pop() would be the last
        new_card.exposed = expose
        if printme:
            print(f"dealing {new_card}")
        hand.cards.append(new_card)

In [7]:
class Hand:
    
    def __init__(self, aces11=True, from_split=None):
        self.aces11 = aces11 # different rules for player hands (ace = 11 ONLY!)
        self.from_split = '' if from_split is None else from_split.face
        self.cards = [] if from_split is None else [ from_split ] # only non-empty after a split
#         self.hittable = True  # hand becomes un-hittable after double-down and after splitting aces
        self.doubled = False
        self.late_surrendered = False
        self.df_surrendered = False
#         self.split_aces = False
        
    def __repr__(self):
        return f"Hand: {'[BUSTED] ' if self.busted else ''}{'[DBL] ' if self.doubled else ''}cards={self.cards}, value={'SOFT-' if self.soft else ''}{self.value}, {'' if self.hittable else 'NOT '}hittable"
    
    @property
    def aces(self):
        return len([ c for c in self.cards if c.face == 'A' if c.exposed])
    
    @property
    def isAA(self):
        return(len(self.cards) == 2 and self.aces == 2)
    
    @property
    def soft(self):
        if self.aces == 0 or self.aces11: 
            return False
        return sum([ (1 if c.face == 'A' else c.value) for c in self.cards ]) < self.value
        
    @property
    def value(self):
        val = sum([ c.value for c in self.cards ])
        if not(self.aces11):
            for a in range(self.aces):
                val = val - 10 if val > 21 else val
        return val        

    @property
    def busted(self):
        # initial AA is an exception to busting, but only before splits
        return self.value > 21 and not(self.isAA and self.from_split == '')
    
    @property
    def blackjack(self):
        return self.value == 21 and len(self.cards) == 2 and self.aces == 1 and self.from_split == '' and not(self.doubled)
    
    @property
    def splittable(self):
        return len(self.cards) == 2 and self.cards[0].value == self.cards[1].value and not(self.isAA) # not(self.split_aces)
    
    @property
    def hittable(self):
        return self.value <= 21 and not(self.doubled) and not(self.split_aces) and not(self.blackjack)
    
    @property
    def split_aces(self):
        return self.from_split == 'A'
    
    
    def split(self):
        if self.splittable:
            return [ Hand(aces11=True, from_split=self.cards[0]), Hand(aces11=True, from_split=self.cards[1]) ]
        else:
            raise Exception("hand not splittable")
            
    def doubledown(self, shoe):
        shoe.pitch(self, printme=False)
        self.doubled = True
        
    def late_surrender(self):
        self.late_surrendered = True
        
    def surrender_after_bust(self):
        self.df_surrendered = True        
        

In [8]:
class Player:
    def __init__(self, name, starting_bankroll):
        self.name = name
        self.starting_bankroll = starting_bankroll
        self.current_bankroll = starting_bankroll
        self.games_played = 0
        self.hands_played = 0
        self.starting_bets = 0
        self.total_bet = 0
        self.current_hands = []
        self.current_bet = 0
        
    def __repr__(self):
        return "".join([f"{self.name}: games {self.games_played}, hands {self.hands_played}, ",
            f"starting_bets {self.starting_bets}, total_bets {self.total_bet} ",
            f"w/l ${self.total_winloss}, h/a {self.house_edge}"])
    
    # def to_dict(self):
    def __iter__(self):
        yield 'name', self.name
        yield 'starting_bankroll', self.starting_bankroll
        yield 'current_bankroll', self.current_bankroll
        yield 'games_played', self.games_played
        yield 'hands_played', self.hands_played
        yield 'starting_bets', self.starting_bets
        yield 'total_bet', self.total_bet
        yield 'winloss', self.starting_bankroll - self.current_bankroll

    
    @property
    def total_winloss(self):
        return self.current_bankroll - self.starting_bankroll
    
    @property
    def house_edge(self):
        return (self.current_bankroll - self.starting_bankroll) / (1 if self.starting_bets == 0 else self.starting_bets)
    
    def update_bankroll(self, bankroll_change, hand_count=1):
        self.current_bankroll = self.current_bankroll + bankroll_change
        self.games_played += 1
        self.hands_played += hand_count
    
    def bet(self, bet_amount):
        self.current_hands = [ Hand() ]
        self.current_bet = bet_amount
        self.starting_bets += bet_amount
        self.total_bet += bet_amount
        
    def early_decisions(self, dealer_exposed, shoe, dealer_blackjack=False):
        for i, hand in enumerate(self.current_hands[:]):  # important to know the index of the hand
            
            # if hand was just split, it needs its second card
            if hand.from_split != '' and len(hand.cards) == 1:
                shoe.pitch(hand)

            if hand.splittable:
                if dealer_exposed in df_split_strategy[hand.cards[0].value]:
                    if len(self.current_hands) < 4:
                        self.total_bet += self.current_bet
                        # squeezing in two new hands in the middle of a list
                        self.current_hands[i:i+1] = hand.split()
                        self.early_decisions(dealer_exposed, shoe, dealer_blackjack) # added a hand, so let's re-run this function
#                         break
                        
            if hand.value in df_double_strategy["soft" if hand.soft else "hard"].keys():
                if dealer_exposed in df_double_strategy["soft" if hand.soft else "hard"][hand.value]:
#                     print(f"doubling with {hand.cards} ({hand.value}) vs dealer {dealer_exposed}")
                    self.total_bet += self.current_bet
                    hand.doubledown(shoe)

            
    def late_decisions(self, dealer_hand, shoe):
        for hand in self.current_hands:
            if hand.busted or hand.blackjack or hand.doubled or hand.split_aces:
                pass
            elif dealer_hand.busted:
                while(hand.value) <= 13 and hand.hittable:
                    shoe.pitch(hand, printme=False)
            else:
                while not(hand.busted) and hand.hittable and hand.value <= dealer_hand.value:
                    shoe.pitch(hand, printme=False)  

        

In [9]:
class Dealer:
    def __init__(self):
        self.hand = None
        
    def reset(self):
        self.hand = Hand(aces11=False)
    
    def expose_all(self):
        [ c.expose() for c in self.hand.cards ]
    
    def play_hand(self, shoe, hit_soft_17=True):
        self.expose_all()
        while (self.hand.value < 17 or (self.hand.value == 17 and self.hand.soft and hit_soft_17)):
            shoe.pitch(self.hand, printme=False)
            
    def peek(self):
        has_blackjack = False
        hidden_card = [c for c in self.hand.cards if not(c.exposed)][0]
        if self.hand.value == 11:
            has_blackjack = hidden_card.face in ['T', 'J', 'Q', 'K']
        elif self.hand.value == 10:
            has_blackjack = hidden_card.face == 'A'
        if has_blackjack:
            self.expose_all()
        return has_blackjack
        

In [10]:
class Game:
    
    def __init__(self, logger=None):
        self.logger = logger or Logger()
        self.shoe = Shoe()
        self.dealer = Dealer()
        self.players = []
    
    def accept_bets(self, players, bets):
        self.players = players
        for p, b in zip(players, bets):
            p.bet(b)
        self.play()

    def play(self):            
        self.reset_all()
        self.deal_first_two()
        self.play_out_game()
            
    def reset_all(self):
        self.dealer.reset()
        for p in self.players:
            p.current_hands = [ Hand() ]
        self.shoe.shuffle_check()
            
    def deal_first_two(self):
        for p in self.players:
            self.shoe.pitch(p.current_hands[0])
        self.shoe.pitch(self.dealer.hand, expose=False)
        for p in self.players:
            self.shoe.pitch(p.current_hands[0])
        self.shoe.pitch(self.dealer.hand, expose=True)
        
    def play_out_game(self):
        dealer_natural = self.dealer.peek()
        
        if not(dealer_natural):

            for p in self.players:
                p.early_decisions(self.dealer.hand.value, self.shoe)
            
            self.dealer.play_hand(self.shoe)
            
            for p in self.players:
                p.late_decisions(self.dealer.hand, self.shoe)         
        
        for p in self.players:
            winloss = sum([self.evaluate_hand(h, self.dealer.hand, p.current_bet) for h in p.current_hands])
            p.update_bankroll(winloss, len(p.current_hands))
                
    
    def evaluate_hand(self, player_hand, dealer_hand, initial_bet, debug=False):
        result, description = ("", "")
        
        if player_hand.blackjack and not(dealer_hand.blackjack):
            result, description = ("player_blackjack", "player BJ")
        elif dealer_hand.blackjack and not(player_hand.blackjack):
            result, description = ("player_loss", "dealer BJ")
        elif dealer_hand.blackjack and player_hand.blackjack:
            result, description = ("push", "player + dealer BJ")
            
        elif player_hand.isAA:
            result, description = ("push", "player_AA")
        elif player_hand.busted and dealer_hand.busted:
            result, description = ("player_loss", "both bust")
        elif (player_hand.busted and not(dealer_hand.busted)): 
            result, description = ("player_loss", "player bust")
        elif dealer_hand.busted and not(player_hand.busted):
            if player_hand.value <= 13 and not(player_hand.doubled or player_hand.split_aces):
                print(player_hand)
                print(dealer_hand)
                raise Exception("player didn't exceed 13 after dealer bust")
            result, description = ("player_win", "dealer bust")
            
        elif player_hand.value < dealer_hand.value and not(player_hand.busted) and not(dealer_hand.busted):
            result, description = ("player_loss", f"dealer={dealer_hand.value}, player={player_hand.value}")
        elif player_hand.value > dealer_hand.value and not(player_hand.busted) and not(dealer_hand.busted):
            result, description = ("player_win", f"player={player_hand.value}, dealer={dealer_hand.value}")
        elif player_hand.value == dealer_hand.value:
            if not(player_hand.doubled or player_hand.split_aces):
                print(player_hand)
                print(dealer_hand)
                raise Exception("push without double-down or split aces")
            result, description = ("push", f"player={player_hand.value}, dealer={dealer_hand.value}")         
        else:
            raise Exception("unknown hand payout")
            
        hand_bet = (2 if player_hand.doubled else 1) * initial_bet
        hand_payout = ((2 if player_hand.doubled else 1) * initial_bet * paytable[result])
        

#         if debug: 
#             print(f'{result}{"-DBL" if player_hand.doubled else ""}{"" if player_hand.from_split == "" else f"-SPLIT:{player_hand.from_split}"} ({description})')
#             print((player_hand.value,
#                dealer_hand.value,
#                '-'.join([str(c) for c in player_hand.cards]),
#                '-'.join([str(c) for c in dealer_hand.cards])))
            
#         self.logger.log_hand(player_hand, dealer_hand, description, hand_bet, hand_payout)
        
#         hand_list.append( (game_count,
#                        player_hand.cards[0].face + player_hand.cards[1].face, 
#                        dealer_hand.cards[1].face, 
#                        player_hand.value,
#                        dealer_hand.value,
#                        '-'.join([str(c) for c in player_hand.cards]),
#                        '-'.join([str(c) for c in dealer_hand.cards]),
#                        win
#                       ) )
        
        return hand_payout - hand_bet
    
            
        
    

In [11]:
class Logger:
    
    def __init__(self, logfile="py_log_test.txt"):
        self.logfile = open(logfile, "w", encoding="utf-8")
        self.game_count = 0
        self.hands = []
        
    def log_hand(self, player_hand, dealer_hand, outcome, bet, payoff):
        self.hands.append(( sum([c.value for c in player_hand.cards[:2]]), dealer_hand.cards[1].face,
            player_hand.cards, dealer_hand.cards, player_hand.value, dealer_hand.value, 
            outcome, bet, payoff, player_hand.doubled, player_hand.from_split))
        if len(self.hands) >= 100:
            self.dump()
            
        
    def dump(self):
        for t in self.hands:  # t for tuple
            self.logfile.write("{};{};{}\n".format(
                '' if t[-1] == '' else f"[SPLIT {t[-1]}]",
                '' if not(t[-2]) else "[DBL]",
                ';'.join(str(s) for s in t[:-2])
            ))
        self.hands.clear()
            
#             self.logfile.write(f"[SPLIT {t[-1]}] {';'.join(str(s) for s in t[:-2])} \n")
            
            
#             if t[-1] == '':
#                 self.logfile.write(';'.join(str(s) for s in t[:-2]) + '\n')
#             else:
#                 self.logfile.write(f"[SPLIT {t[-1]}] {';'.join(str(s) for s in t[:-2])} \n")

    def close(self):
        self.dump()
        self.logfile.close()

### first run: we just run a simple loop, one player, with no concurrency

In [12]:
def single_loop(players, bets, games_to_sim, loop_id="", detail_print=True):
    game = Game()
    
    for i in range(games_to_sim):
        if i % 25000 == 0:
            print(f"[{loop_id}] ... game number {i} ...")
        game.accept_bets(players, bets) # all_players, bet_amounts)
    game.logger.close()
    
    player_results = players
    
    if detail_print:
        for p in player_results:
            print(p)
    
    print("".join([f"total: games {sum([p.games_played for p in player_results])}, hands {sum([p.hands_played for p in player_results])}, ", 
              f"starting_bets ${sum([p.starting_bets for p in player_results])}, total_bets ${sum([p.total_bet for p in player_results])}, ",
              f"w/l ${sum([p.total_winloss for p in player_results])}, ",
              f"h/a {sum([p.total_winloss for p in player_results]) / sum([p.starting_bets for p in player_results]) }"] ) )

    return player_results



In [13]:
games_to_sim = 500000

start = timer()
loop_results = single_loop([Player("kevin", 1000000)], [100], games_to_sim)
end = timer()
print(f'time elapsed = {f"{end - start} seconds" if end - start < 300 else f"{(end - start) / 60} minutes"}')


[] ... game number 0 ...
[] ... game number 25000 ...
[] ... game number 50000 ...
[] ... game number 75000 ...
[] ... game number 100000 ...
[] ... game number 125000 ...
[] ... game number 150000 ...
[] ... game number 175000 ...
[] ... game number 200000 ...
[] ... game number 225000 ...
[] ... game number 250000 ...
[] ... game number 275000 ...
[] ... game number 300000 ...
[] ... game number 325000 ...
[] ... game number 350000 ...
[] ... game number 375000 ...
[] ... game number 400000 ...
[] ... game number 425000 ...
[] ... game number 450000 ...
[] ... game number 475000 ...
kevin: games 500000, hands 508349, starting_bets 50000000, total_bets 55827000 w/l $-325100.0, h/a -0.006502
total: games 500000, hands 508349, starting_bets $50000000, total_bets $55827000, w/l $-325100.0, h/a -0.006502
time elapsed = 51.789939600000004 seconds


In [14]:
def get_players():
    return [
        Player("aaron", 1000000),
        Player("bobby", 1000000),
        Player("chuck", 1000000),
        Player("danny", 1000000),
        Player("eddie", 1000000)]
bet_amounts = [100, 100, 100, 100, 100]

if len(get_players()) != len(bet_amounts):
    raise Exception("different quantity of players and bets")


### next: place 5 players at the table, but still just a simple loop

In [15]:
games_to_sim = 100000

start = timer()
loop_results = single_loop(get_players(), bet_amounts, games_to_sim)
end = timer()
print(f'time elapsed = {f"{end - start} seconds" if end - start < 300 else f"{(end - start) / 60} minutes"}')


[] ... game number 0 ...
[] ... game number 25000 ...
[] ... game number 50000 ...
[] ... game number 75000 ...
aaron: games 100000, hands 101714, starting_bets 10000000, total_bets 11160700 w/l $-69600.0, h/a -0.00696
bobby: games 100000, hands 101692, starting_bets 10000000, total_bets 11178800 w/l $-41600.0, h/a -0.00416
chuck: games 100000, hands 101574, starting_bets 10000000, total_bets 11142000 w/l $-59200.0, h/a -0.00592
danny: games 100000, hands 101640, starting_bets 10000000, total_bets 11152800 w/l $-69950.0, h/a -0.006995
eddie: games 100000, hands 101605, starting_bets 10000000, total_bets 11140800 w/l $-43500.0, h/a -0.00435
total: games 500000, hands 508225, starting_bets $50000000, total_bets $55775100, w/l $-283850.0, h/a -0.005677
time elapsed = 35.541495 seconds


### now we use `joblib` to implement concurency / parallelism
### for max speed, we will connect to a 32-core (or more) AWS EC2 server and run this notebook there

In [16]:
    
######
#
#  we're using loops and lists of lists in order to take advantage of parallelism, 
#    running each loop on different threads/processes simultaneously
#  benefit is faster running, but then we have to re-combine the output data and the individual log files at the end
#
######

loops = 20
games_per_loop = 5000


start = timer()
loop_results = Parallel(n_jobs=8)(delayed(single_loop)(get_players(), bet_amounts, games_per_loop, i) for i in range(loops))
end = timer()
print(f'time elapsed = {f"{end - start} seconds" if end - start < 300 else f"{(end - start) / 60} minutes"}')

# transpose the result lists so all results of a same player are in the same list
# see https://stackoverflow.com/questions/6473679/transpose-list-of-lists
combined_player_results = list(map(list, zip(*loop_results)))


def create_player_totals(result_list):
    player_counter = collections.Counter()
    for result in result_list:
        player_counter.update(dict(result)) # Counter.update sums same values in a list of dicts
    player_result = dict(player_counter)
    player_result['house_edge'] = player_result['winloss'] / player_result['starting_bets']
    
    print("".join([f"{player_result['name'][:5]}: games {player_result['games_played']}, hands {player_result['hands_played']}, ", 
              f"starting_bets ${player_result['starting_bets']}, total_bets ${player_result['total_bet']}, ",
              f"w/l ${player_result['winloss']}, h/a {player_result['house_edge']}"]))
    return player_result



player_totals = [create_player_totals(p) for p in combined_player_results]



total_counter = collections.Counter()
for pt in player_totals:
    total_counter.update(pt)
combined_result = dict(total_counter)
combined_result['name'] = "all_players"
combined_result['house_edge'] = combined_result['winloss'] / combined_result['starting_bets']

print("".join([f"{combined_result['name']}: games {combined_result['games_played']}, hands {combined_result['hands_played']}, ", 
          f"starting_bets ${combined_result['starting_bets']}, total_bets ${combined_result['total_bet']}, ",
          f"w/l ${combined_result['winloss']}, h/a {combined_result['house_edge']}"]))

# print(complete_result)


# with open("py_log_file_combined.txt", "wb") as endfile:
#     for i in range(10):
#         with open(f"py_log_file_{i}.txt", "rb") as section:
#             shutil.copyfileobj(section, endfile)

time elapsed = 24.8391941 seconds
aaron: games 100000, hands 101610, starting_bets $10000000, total_bets $11144900, w/l $99900.0, h/a 0.00999
bobby: games 100000, hands 101679, starting_bets $10000000, total_bets $11157700, w/l $103000.0, h/a 0.0103
chuck: games 100000, hands 101687, starting_bets $10000000, total_bets $11177800, w/l $49000.0, h/a 0.0049
danny: games 100000, hands 101616, starting_bets $10000000, total_bets $11152000, w/l $54900.0, h/a 0.00549
eddie: games 100000, hands 101685, starting_bets $10000000, total_bets $11150800, w/l $75350.0, h/a 0.007535
all_players: games 500000, hands 508277, starting_bets $50000000, total_bets $55783200, w/l $382150.0, h/a 0.007643
